In [1]:
# %% Imports + your helpers (paste your own versions if already defined above)
import numpy as np

import sys
import os

# Add parent folder (one level up from notebook) to Python path
sys.path.append(os.path.abspath(".."))

# If your function lives in another file, import it here:
from amp_denoising_functions import beta_tilde, c_1, c_2, p_out, Z_out, f_out, df_out, f_Q, gamma_matrix, integral_parameters
from lambda_nu_integrals import int_1, int_2, int_3, int_4, int_5
from state_evolution import generate_latents


In [2]:
# %% Monte-Carlo estimator of the exact expression in your image
def p_out_mc(beta_u, beta_v, gamma, delta, y, z, Q, n_samples=200_000, seed=0):
    """
    Direct MC of the expectation over (lambda, nu) appearing in the displayed P_out formula.
    """
    if y == -1:
        return 0.5

    rng = np.random.default_rng(seed)

    bt = beta_tilde(beta_v)
    c2 = c_2(bt, Q[1,1])
    # guard against singular c2
    if np.isclose(c2, 0.0):
        c2 = np.copysign(max(abs(c2), 1e-12), c2)
    c1v = c_1(c2, bt)

    Q11, Q12, Q22 = Q[0,0], Q[0,1], Q[1,1]
    z1, z2 = z

    lam, nu = generate_latents(n_samples, gamma, delta)

    # exponent terms (vectorized)
    expo = (
        -0.5*beta_u * (Q11 + c1v*Q12**2) * lam**2
        + np.sqrt(beta_u) * lam * (z1 + c1v*Q12*z2)
        + (np.sqrt(beta_v)/c2) * nu * z2
        - (np.sqrt(beta_u*beta_v)/c2) * lam * nu * Q12
    )

    # numerically-stable mean of exp(expo)
    m = expo.max()
    inner = np.exp(expo - m).mean() * np.exp(m)

    pref = (1/(2*np.abs(c2))) * np.exp(-0.5*beta_v*Q22) * np.exp(-0.5*c1v*z2**2)
    return pref * inner

In [3]:
# %% Convenience: random SPD(2) and quick runner
def random_spd_2x2(rng, jitter=0.25):
    A = rng.normal(size=(2,2))
    S = A @ A.T
    S += jitter * np.eye(2)
    return S

def compare_once(beta_u, beta_v, gamma, delta, Q, z, n_samples=200_000, seed=123):
    mc = p_out_mc(beta_u, beta_v, gamma, delta, y=1, z=z, Q=Q, n_samples=n_samples, seed=seed)
    an = p_out(beta_u, beta_v, gamma, delta, y=1, z=z, Q=Q)
    abs_err = abs(an - mc)
    rel_err = abs_err / max(abs(mc), 1e-15)
    return an, mc, abs_err, rel_err


In [4]:
# %% Run a few randomized trials + edge case (Q12=0)
rng = np.random.default_rng(7)

beta_u = 1
beta_v = 2
gamma  = 0
delta  = 1

def run_trials(n_trials=8, n_samples=250_000):
    stats = []
    for i in range(n_trials):
        Q = random_spd_2x2(rng)
        z = rng.normal(size=2)

        an, mc, ae, re = compare_once(
            beta_u, beta_v, gamma, delta, Q, z,
            n_samples=n_samples, seed=rng.integers(1<<31)
        )
        stats.append((ae, re))
        print(f"Trial {i+1:02d}")
        print(f"  Q =\n{Q}")
        print(f"  z = {z}")
        print(f"  analytic p_out = {an:.6e}")
        print(f"  Monte Carlo    = {mc:.6e}")
        print(f"  abs err = {ae:.3e}, rel err = {re:.3e}")
        print("-"*60)

    # edge case 1: Q12 = 0
    Qd = random_spd_2x2(rng)
    Qd[0,1] = Qd[1,0] = 0.0
    z = rng.normal(size=2)
    an, mc, ae, re = compare_once(beta_u, beta_v, gamma, delta, Qd, z, n_samples=n_samples)
    stats.append((ae, re))
    print("Edge case: Q12 = 0")
    print(f"  Q =\n{Qd}")
    print(f"  z = {z}")
    print(f"  analytic p_out = {an:.6e}")
    print(f"  Monte Carlo    = {mc:.6e}")
    print(f"  abs err = {ae:.3e}, rel err = {re:.3e}")
    print("="*60)

    abs_errs = np.array([s[0] for s in stats])
    rel_errs = np.array([s[1] for s in stats])

    print(f"Trials: {len(stats)}")
    print(f"Abs error  — median: {np.median(abs_errs):.3e} | max: {np.max(abs_errs):.3e}")
    print(f"Rel error  — median: {np.median(rel_errs):.3e} | max: {np.max(rel_errs):.3e}")

run_trials()


Trial 01
  Q =
[[ 0.33925041 -0.26639757]
 [-0.26639757  1.11830539]]
  z = [-0.45467079 -0.99164655]
  analytic p_out = 7.008756e-01
  Monte Carlo    = 6.999681e-01
  abs err = 9.075e-04, rel err = 1.296e-03
------------------------------------------------------------
Trial 02
  Q =
[[ 2.28844416 -1.07267337]
 [-1.07267337  0.87493434]]
  z = [0.35688701 0.10541425]
  analytic p_out = 2.421703e-01
  Monte Carlo    = 2.427582e-01
  abs err = 5.879e-04, rel err = 2.422e-03
------------------------------------------------------------
Trial 03
  Q =
[[ 1.11662645 -0.60763668]
 [-0.60763668  2.54035928]]
  z = [-0.45761576 -1.90122274]
  analytic p_out = 3.401268e-04
  Monte Carlo    = 3.408228e-04
  abs err = 6.960e-07, rel err = 2.042e-03
------------------------------------------------------------
Trial 04
  Q =
[[3.69725579 2.27052875]
 [2.27052875 1.93000494]]
  z = [ 0.15675109 -0.18693094]
  analytic p_out = 3.150651e-01
  Monte Carlo    = 3.175290e-01
  abs err = 2.464e-03, rel err

## is my derived P_out correct though? it was a long derivation. let's compare it with definition

In [5]:
# %% Parameters, Q and its Cholesky S
beta_u = 1.0
beta_v = 2.0
gamma  = 1.0
delta  = 0.0

In [6]:
# %% Define test functions h(X, Y)
def h_list():
    funcs = []

    # means (ignore y)
    funcs.append(("x1", lambda x, y: x[:, 0]))
    funcs.append(("x2", lambda x, y: x[:, 1]))

    # covariances and cross terms
    funcs.append(("x1^2", lambda x, y: x[:, 0]**2))
    funcs.append(("x2^2", lambda x, y: x[:, 1]**2))
    funcs.append(("x1*x2", lambda x, y: x[:, 0]*x[:, 1]))

    # y-coupled moments
    funcs.append(("y*x1", lambda x, y: y * x[:, 0]))
    funcs.append(("y*x2", lambda x, y: y * x[:, 1]))

    # non-linear probes
    funcs.append(("sin(x1)", lambda x, y: np.sin(x[:, 0])))
    funcs.append(("exp(0.2*x2)", lambda x, y: np.exp(0.2 * x[:, 1])))

    # indicator(y=1)
    funcs.append(("1{y=1}", lambda x, y: (y == 1).astype(float)))
    return funcs


In [7]:
from amp_experiment import generate_data

In [8]:
def generate_data(n, d, beta_u, beta_v, gamma=0, delta=1):
    u_star = np.random.randn(d)
    u_star = u_star/np.sqrt(d)
    v_star = np.random.randn(d)
    v_star = v_star/np.sqrt(d)

    y = np.random.choice([-1, 1], size=n, p=[1/2, 1/2])

    X = np.zeros((n, d))

    lambda_vals, nu_vals = generate_latents(n, gamma=gamma, delta=delta)
    S = np.eye(d) - beta_v/(1+beta_v+np.sqrt(1+beta_v))*np.outer(v_star, v_star)

    X = np.sqrt(beta_u)*np.outer(lambda_vals, u_star) + (np.sqrt(beta_v)*np.outer(nu_vals, v_star) + np.random.randn(n, d)) @ S.T

    X[y==-1] = np.random.randn(np.sum(y==-1), d)

    return X, y, u_star, v_star

In [9]:
N_model = 200000   # samples from true model
d = 2

# Left: direct model samples
X_model, Y_model, u_star, v_star = generate_data(N_model, d=d, beta_u=beta_u, beta_v=beta_v, gamma=gamma, delta=delta)

W = np.column_stack((u_star, v_star)) # shape (d, 2)
Q = W.T @ W
print(Q)

X_change = np.random.randn(N_model, d)
Y_base = np.random.choice([-1, 1], size=N_model, p=[1/2, 1/2])

def p_out_vec(beta_u, beta_v, gamma, delta, y, Z, Q):
    Z = np.asarray(Z)
    if Z.ndim == 1:            # (2,) -> (1,2)
        Z = Z[None, :]
    assert Z.shape[1] == 2
    out = np.fromiter(
        (p_out(beta_u, beta_v, gamma, delta, y, Z[i], Q) for i in range(Z.shape[0])),
        dtype=float, count=Z.shape[0]
    )
    return out


# Normalization checks
# 1) E[P_out(1, Z, Q)] ≈ 1/2
mask_y1 = (Y_base == 1)
if np.any(mask_y1):
    # compute directly from P_out (no doubling)
    z = X_change[mask_y1, :] @ W
    print(z.shape)
    P_vals = np.array(p_out_vec(beta_u, beta_v, gamma, delta, 1, z, Q))
    print(P_vals.shape)
    print(P_vals.min(), P_vals.max())
    print("Check: E[P_out(y=1,Z,Q)] ≈ 1/2  →", np.mean(P_vals))


[[ 0.84159287 -0.94603698]
 [-0.94603698  1.58830906]]
(99859, 2)
(99859,)
1.357157441066402e-09 7.083157755206577
Check: E[P_out(y=1,Z,Q)] ≈ 1/2  → 0.49989354883398035


In [10]:
# ========= Expectation tests with change of measure =========

funcs = h_list()

# Precompute Z and probabilities on the base (change) distribution
Z_change = X_change @ W                      # (N, 2)
p = p_out_vec(beta_u, beta_v, gamma, delta, 1, Z_change, Q)   # (N,)
one_minus_p = 1.0 - p

# Convenience y arrays
Y_pos = np.ones(N_model, dtype=int)
Y_neg = -np.ones(N_model, dtype=int)



results = []
for name, h in funcs:
    # Direct mean under the true model
    h_model = h(X_model, Y_model)
    mean_direct = float(np.mean(h_model))
    se_direct = float(np.std(h_model, ddof=1) / np.sqrt(h_model.size))

    # Change-of-measure mean using base X_change and p_out
    h_pos = h(X_change, Y_pos)   # h(x, +1)
    h_neg = h(X_change, Y_neg)   # h(x, -1)
    h_cm  = h_pos * p + h_neg * 1/2
    mean_cm = float(np.mean(h_cm))
    se_cm = float(np.std(h_cm, ddof=1) / np.sqrt(h_cm.size))

    err = mean_cm - mean_direct
    results.append((name, mean_direct, se_direct, mean_cm, se_cm, err))

# Pretty print
w = 12
print(f"{'h':<10} {'direct':>{w}} {'(±SE)':>8} {'change-meas':>{w}} {'(±SE)':>8} {'diff(cm-direct)':>{w}}")
for name, md, sed, mc, sec, err in results:
    print(f"{name:<10} {md:>{w}.6f} {sed:>8.4f} {mc:>{w}.6f} {sec:>8.4f} {err:>{w}.6f}")

# Extra: normalization sanity checks
print("\nSanity checks:")
print("  E_model[1{y=1}]      ≈", np.mean(Y_model == 1))
print("  E_change[p_out(1|Z)] ≈", float(np.mean(p)))


h                direct    (±SE)  change-meas    (±SE) diff(cm-direct)
x1            -0.001494   0.0020     0.000571   0.0020     0.002065
x2            -0.000357   0.0022     0.001801   0.0025     0.002158
x1^2           0.797592   0.0026     0.799070   0.0023     0.001477
x2^2           1.000302   0.0032     0.996586   0.0041    -0.003716
x1*x2         -0.231717   0.0022    -0.231272   0.0026     0.000446
y*x1           0.002758   0.0020    -0.001884   0.0010    -0.004642
y*x2           0.000781   0.0022     0.000643   0.0011    -0.000137
sin(x1)       -0.000776   0.0014    -0.000122   0.0015     0.000655
exp(0.2*x2)     1.020139   0.0005     1.019674   0.0010    -0.000464
1{y=1}         0.500125   0.0011     0.499159   0.0008    -0.000966

Sanity checks:
  E_model[1{y=1}]      ≈ 0.500125
  E_change[p_out(1|Z)] ≈ 0.4991589734602604


## testing Z_out, f_out, df_out, f_Q

In [11]:
def Z_out_MC(beta_u, beta_v, gamma, delta, y, omega, Q, V, n_samples=200_000):
    """
    Monte Carlo estimate of Z_out(y, omega, V) = E_Z[P_out(y|Z,Q)]
    where Z ~ N(omega, V).
    """
    Z_samples = np.random.multivariate_normal(mean=omega, cov=V, size=n_samples)
    P_vals = np.array(p_out_vec(beta_u, beta_v, gamma, delta, y, Z_samples, Q))
    mean_val = np.mean(P_vals)
    stderr = np.std(P_vals) / np.sqrt(n_samples)
    return mean_val, stderr


def Z_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    """
    Analytic version using your formula and functions c_1, c_2, gamma_matrix, int_1.
    """
    if y == -1:
        return 1/2
    
    # Decompose omega
    omega1, omega2 = omega

    # Compute c1, c2
    beta_tilde_v = beta_tilde(beta_v)
    c2 = c_2(beta_tilde_v, Q[1,1])
    c1 = c_1(c2, beta_tilde_v)

    # Gamma matrix
    Gamma = gamma_matrix(c1)

    # Matrices M, N
    M = np.linalg.inv(np.linalg.inv(V) + Gamma)
    N = np.linalg.inv(V) @ M

    # Coefficients a,b,c,d
    a = beta_u*(Q[0,0]+c1*Q[0,1]**2-M[0,0]-2*c1*Q[0,1]*M[0,1]-c1**2*Q[0,1]**2*M[1,1])
    b = np.sqrt(beta_u)*(omega1*N[0,0]+omega2*N[1,0]+c1*Q[0,1]*(omega1*N[0,1]+omega2*N[1,1]))
    c = np.sqrt(beta_v)/c2*(omega1*N[0,1]+omega2*N[1,1])
    d = np.sqrt(beta_u*beta_v)/c2 * (-Q[0,1]+M[0,1]+c1*Q[0,1]*M[1,1])

    return Z_out(beta_v=beta_v, gamma=gamma, delta=delta, c1=c1, c2=c2, y=1, omega=omega, Q=Q, M=M, N=N, a=a, b=b, c=c, d=d)


# === Example random test ===
beta_u, beta_v = 1.0, 2.0
gamma, delta = 1.0, 0.0
y = 1

# Random SPD Q and V
# A = np.random.randn(2, 2)
# Q = A.T @ A + 0.5*np.eye(2)
# B = np.random.randn(2, 2)
# V = B.T @ B + 0.3 * np.eye(2)

# Q = 0.3*np.eye(2)
Q = np.array([[0.6, -0.5], [-0.5, 0.2]])
#V = 0.4*np.eye(2)
V = np.array([[0.6, -0.3], [-0.3, 0.5]])

omega = np.random.randn(2) * 0.2

# Monte Carlo
mc_mean, mc_se = Z_out_MC(beta_u, beta_v, gamma, delta, y, omega, Q, V)
print(f"Monte Carlo Z_out = {mc_mean:.6f} ± {mc_se:.6f}")

# Analytic
analytic_val = Z_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V)
print(f"Analytic Z_out    = {analytic_val:.6f}")

# Comparison
rel_err = abs(analytic_val - mc_mean) / (mc_mean + 1e-12)
print(f"Relative error: {rel_err:.3%}")



Monte Carlo Z_out = 0.695489 ± 0.000844
Analytic Z_out    = 0.695109
Relative error: 0.055%


In [12]:


# ----- helpers: build params and analytic Z_out -----

def _build_params(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    bt = beta_tilde(beta_v)                 # = beta_v/(1+beta_v+sqrt(1+beta_v))
    c2 = c_2(bt, Q[1,1])
    c1 = c_1(c2, bt)

    Gamma = gamma_matrix(c1)                # 2x2, only [1,1]=c1
    Vinv  = np.linalg.inv(V)
    M = np.linalg.inv(Vinv + Gamma)
    N = Vinv @ M

    omega1, omega2 = omega
    a = beta_u * ( Q[0,0] + c1*Q[0,1]**2 - M[0,0] - 2*c1*Q[0,1]*M[0,1] - (c1**2)*(Q[0,1]**2)*M[1,1] )
    b = np.sqrt(beta_u) * ( omega1*N[0,0] + omega2*N[1,0] + c1*Q[0,1]*(omega1*N[0,1] + omega2*N[1,1]) )
    c = np.sqrt(beta_v)/c2 * ( omega1*N[0,1] + omega2*N[1,1] )
    d = np.sqrt(beta_u*beta_v)/c2 * ( -Q[0,1] + M[0,1] + c1*Q[0,1]*M[1,1] )

    # your confirmed analytic Z_out:
    Zout_analytic = Z_out(beta_v, gamma, delta, c1, c2, y, omega, Q, M, N, a, b, c, d)

    return c1, c2, M, N, a, b, c, d, Vinv, Zout_analytic, Gamma

# ----- MC g_out using analytic Z_out in the denominator -----

def g_out_MC_using_ZoutAnalytic(beta_u, beta_v, gamma, delta, y, omega, Q, V, n_samples=200_000, n_boot=0):
    c1, c2, M, N, a, b, c, d, Vinv, Zout_analytic, Gamma = _build_params(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )

    # MC numerator: E[(Z-omega) P_out(y|Z,Q)]
    Z = np.random.multivariate_normal(mean=omega, cov=V, size=n_samples)
    P = p_out_vec(beta_u, beta_v, gamma, delta, y, Z, Q)            # (n,)
    num_hat = ((Z - omega) * P[:, None]).mean(axis=0)               # (2,)

    g_mc = Vinv @ (num_hat / (Zout_analytic + 1e-15))               # (2,)

    if n_boot > 0:
        n = len(P)
        boots = []
        for _ in range(n_boot):
            idx = np.random.randint(0, n, size=n)
            num_b = ((Z[idx] - omega) * P[idx, None]).mean(axis=0)
            boots.append(Vinv @ (num_b / (Zout_analytic + 1e-15)))
        boots = np.vstack(boots)
        se = boots.std(axis=0, ddof=1)
        return g_mc, se
    else:
        return g_mc, None

# ----- Analytic f_out wrapper (adapt call to your signature) -----

def g_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    c1, c2, M, N, a, b, c, d, Vinv, Zout_analytic, Gamma = _build_params(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )
    # call YOUR analytic function:
    return f_out(beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)

# ----- Quick test -----
# --- Final comparison run (using your Q, V) ---

np.random.seed(0)

beta_u, beta_v = 1.0, 2.0
gamma, delta   = 1.0, 0.0
omega = np.random.randn(2) * 0.2

# Q = 0.3*np.eye(2)
Q = np.array([[0.8, 0], [0, 0.2]])
#V = 0.4*np.eye(2)
V = np.array([[0.6, 0], [0, 0.5]])

for y in (1, -1):
    g_mc, se = g_out_MC_using_ZoutAnalytic(
        beta_u, beta_v, gamma, delta, y, omega, Q, V,
        n_samples=2000000, n_boot=100
    )
    g_an = g_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V)

    print(f"y={y}")
    print(f"  g_out MC (using analytic Z_out): {g_mc}  ± {se}")
    print(f"  g_out analytic (f_out)         : {g_an}")
    print(f"  diff (MC - analytic)           : {g_mc - g_an}\n")



y=1
  g_out MC (using analytic Z_out): [0.35139801 0.29214659]  ± [0.00242407 0.00154611]
  g_out analytic (f_out)         : [0.35034445 0.29044108]
  diff (MC - analytic)           : [0.00105356 0.0017055 ]

y=-1
  g_out MC (using analytic Z_out): [-0.00042231  0.00137012]  ± [0.00101982 0.00100765]
  g_out analytic (f_out)         : [0. 0.]
  diff (MC - analytic)           : [-0.00042231  0.00137012]



In [13]:


# ===== MC estimator for ∂_w g_out using analytic Z_out and f_out =====

def dg_out_MC_using_Zout_fout(beta_u, beta_v, gamma, delta, y, omega, Q, V,
                              n_samples=200_000, n_boot=100):
    """
    Returns:
      J_mc: 2x2 Monte Carlo estimate of ∂_w g_out
      SE:   2x2 bootstrap standard errors (if n_boot>0) else None
    """
    # Build params and analytic pieces (reuse your helper)
    c1, c2, M, N, a, b, c, d, Vinv, Zout_an, Gamma = _build_params(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )
    # analytic g_out (vector)
    g_an = f_out(beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)

    # MC numerator: E[ (Z-w)(Z-w)^T * P_out(y|Z,Q) ]
    Z = np.random.multivariate_normal(mean=omega, cov=V, size=n_samples)
    P = p_out_vec(beta_u, beta_v, gamma, delta, y, Z, Q)  # (n,)

    Zm = Z - omega                                        # (n,2)
    outer = Zm[:, :, None] * Zm[:, None, :]               # (n,2,2)
    S_hat = (outer * P[:, None, None]).mean(axis=0)       # (2,2)

    # Jacobian via the formula
    J_mc = Vinv @ (S_hat / (Zout_an + 1e-15)) @ Vinv - Vinv - np.outer(g_an, g_an)

    SE = None
    if n_boot > 0:
        n = len(P)
        boots = np.empty((n_boot, 2, 2))
        idxs = np.random.randint(0, n, size=(n_boot, n))
        for bti in range(n_boot):
            idx = idxs[bti]
            Zm_b = Zm[idx]
            P_b  = P[idx]
            outer_b = Zm_b[:, :, None] * Zm_b[:, None, :]
            S_b = (outer_b * P_b[:, None, None]).mean(axis=0)
            boots[bti] = Vinv @ (S_b / (Zout_an + 1e-15)) @ Vinv - Vinv - np.outer(g_an, g_an)
        SE = boots.std(axis=0, ddof=1)

    return J_mc, SE

# ===== Analytic wrapper (call YOUR closed-form Jacobian) =====
# Rename `df_out` in the next line if your function name differs.
def dg_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    c1, c2, M, N, a, b, c, d, Vinv, Zout_an, Gamma = _build_params(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )
    fout = f_out(beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)
    # Your analytic Jacobian (2x2):
    J_an = df_out(fout, beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)
    return J_an

# ===== Example run with your Q and V =====

np.random.seed(0)

beta_u, beta_v = 1.0, 2.0
gamma,  delta  = 1.0, 0.0
omega = np.random.randn(2) * 0.2

# Q = 0.3*np.eye(2)
Q = np.array([[0.8, -0.5], [-0.5, 0.7]])
#V = 0.4*np.eye(2)
V = np.array([[0.6, -0.1], [-0.1, 0.5]])

for y in (1, -1):
    J_mc, SE = dg_out_MC_using_Zout_fout(beta_u, beta_v, gamma, delta, y, omega, Q, V,
                                         n_samples=200_000, n_boot=200)
    J_an = dg_out_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V)

    print(f"\ny={y}")
    print("∂_w g_out  (MC):")
    print(J_mc)
    if SE is not None:
        print("± SE:")
        print(SE)
    print("∂_w g_out (analytic):")
    print(J_an)
    print("diff (MC - analytic):")
    print(J_mc - J_an)

    # Optional summary stats
    frob = np.linalg.norm(J_mc - J_an, 'fro')
    zscores = (J_mc - J_an) / (SE + 1e-15) if SE is not None else None
    print(f"Frobenius diff: {frob:.4e}")
    if zscores is not None:
        print("z-scores per entry:")
        print(zscores)



y=1
∂_w g_out  (MC):
[[ 1.00584269  0.50729258]
 [ 0.50729258 -0.12646725]]
± SE:
[[0.0492043  0.01125882]
 [0.01125882 0.00830826]]
∂_w g_out (analytic):
[[ 0.99008992  0.52687647]
 [ 0.52687647 -0.12090109]]
diff (MC - analytic):
[[ 0.01575278 -0.01958389]
 [-0.01958389 -0.00556616]]
Frobenius diff: 3.2345e-02
z-scores per entry:
[[ 0.32015042 -1.73942642]
 [-1.73942642 -0.66995445]]

y=-1
∂_w g_out  (MC):
[[0.00452693 0.00841035]
 [0.00841035 0.0036401 ]]
± SE:
[[0.00543627 0.00411702]
 [0.00411702 0.00679724]]
∂_w g_out (analytic):
[[0. 0.]
 [0. 0.]]
diff (MC - analytic):
[[0.00452693 0.00841035]
 [0.00841035 0.0036401 ]]
Frobenius diff: 1.3237e-02
z-scores per entry:
[[0.83272764 2.04282524]
 [2.04282524 0.53552594]]


define analytical partial_Q P_out for intermediate testing of f_Q

In [43]:
def partial_Q_p_out(beta_u, beta_v, gamma, delta, y, z, Q, beta_tilde_v, c1, c2):
    if y == -1:
        return np.zeros((2,2))
    if y == 1:
        a = beta_u*(Q[0,0]+c1*Q[0,1]**2)
        b = np.sqrt(beta_u)*(z[0]+c1*Q[0,1]*z[1])
        c = np.sqrt(beta_v)/c2*z[1]
        d = -np.sqrt(beta_u*beta_v)/c2*Q[0,1]
        D = np.zeros((2,2))
        D[0,0] = 1/2*beta_u*int_5(a, b, c, d, gamma, delta)
        D[1,1] = (1/2*beta_v-beta_tilde_v/c2 +1/2*z[1]**2*beta_tilde_v**2/c2**2*(1+2/c2))*int_1(a, b, c, d, gamma, delta) - np.sqrt(beta_u)*beta_tilde_v**2/c2**2*(1+2/c2)*Q[0,1]*z[1]*int_2(a, b, c, d, gamma, delta) - beta_tilde_v*np.sqrt(beta_v)*z[1]/c2**2*int_3(a, b, c, d, gamma, delta) + beta_tilde_v*np.sqrt(beta_u*beta_v)/c2**2*Q[0,1]*int_4(a, b, c, d, gamma, delta) + 1/2*beta_u*beta_tilde_v**2/c2**2*(1+2/c2)*Q[0,1]**2*int_5(a, b, c, d, gamma, delta)
        D[0,1] = np.sqrt(beta_u*beta_v)/c2*int_4(a, b, c, d, gamma, delta) + beta_u*c1*Q[0,1]*int_5(a, b, c, d, gamma, delta) - np.sqrt(beta_u)*c1*z[1]*int_2(a, b, c, d, gamma, delta)
        D[1,0] = D[0,1]

        return -1/(2*np.abs(c2))*np.exp(-1/2*beta_v*Q[1,1])*np.exp(-1/2*c1*z[1]**2)*D

In [38]:
def partial_Q_p_out(beta_u, beta_v, gamma, delta, y, z, Q, beta_tilde_v, c1, c2):
    if y == -1:
        return np.zeros((2,2))
    if y == 1:
        a = beta_u*(Q[0,0]+c1*Q[0,1]**2)
        b = np.sqrt(beta_u)*(z[0]+c1*Q[0,1]*z[1])
        c = np.sqrt(beta_v)/c2*z[1]
        d = -np.sqrt(beta_u*beta_v)/c2*Q[0,1]
        D = np.zeros((2,2))
        D[0,0] = 1/2*beta_u*int_5(a, b, c, d, gamma, delta)
        D[0,1] = np.sqrt(beta_u*beta_v)/c2*int_4(a, b, c, d, gamma, delta) + beta_u*c1*Q[0,1]*int_5(a, b, c, d, gamma, delta) - np.sqrt(beta_u)*c1*z[1]*int_2(a, b, c, d, gamma, delta)
        D[1,0] = D[0,1]

        A = (1/2*beta_v-beta_tilde_v/c2)*int_1(a, b, c, d, gamma, delta)
        B = 1/2*z[1]**2*beta_tilde_v**2/c2**2*(1+2/c2)*int_1(a, b, c, d, gamma, delta)
        C = - beta_tilde_v*np.sqrt(beta_v)*z[1]/c2**2*int_3(a, b, c, d, gamma, delta) 
        D_term = - np.sqrt(beta_u)*beta_tilde_v**2/c2**2*(1+2/c2)*Q[0,1]*z[1]*int_2(a, b, c, d, gamma, delta)
        E = beta_tilde_v*np.sqrt(beta_u*beta_v)/c2**2*Q[0,1]*int_4(a, b, c, d, gamma, delta)
        F = 1/2*beta_u*beta_tilde_v**2/c2**2*(1+2/c2)*Q[0,1]**2*int_5(a, b, c, d, gamma, delta)

        D[1,1] = A + B + C + D_term + E + F
       

        return -1/(2*np.abs(c2))*np.exp(-1/2*beta_v*Q[1,1])*np.exp(-1/2*c1*z[1]**2)*D

In [45]:
# --- helpers for SPD-safe central differences on symmetric Q ---

def _spd_safe_eps(Q, B, eps):
    # shrink eps if Q ± eps B is not SPD
    for _ in range(10):
        lam_min_p = np.min(np.linalg.eigvalsh(Q + eps * B))
        lam_min_m = np.min(np.linalg.eigvalsh(Q - eps * B))
        if lam_min_p > 0 and lam_min_m > 0:
            return eps
        eps *= 0.5
    return eps  # last resort

def _fd_dir_p_out(beta_u, beta_v, gamma, delta, y, z, Q, B, eps):
    eps = _spd_safe_eps(Q, B, eps)
    Pp = p_out(beta_u, beta_v, gamma, delta, y, z, Q + eps * B)
    Pm = p_out(beta_u, beta_v, gamma, delta, y, z, Q - eps * B)
    return (Pp - Pm) / (2.0 * eps)

def _numeric_grad_wrt_Q(beta_u, beta_v, gamma, delta, y, z, Q, eps=2e-4):
    """
    Returns a 2x2 symmetric matrix G such that
    directional derivative in direction B is <G,B>.
    Uses basis B11, B22, B12 where B12 = 0.5*[[0,1],[1,0]].
    """
    B11 = np.array([[1.0, 0.0],[0.0, 0.0]])
    B22 = np.array([[0.0, 0.0],[0.0, 1.0]])
    B12 = np.array([[0.0, 1.0],[0.0, 0.0]])  # ensures <G,B12> = G12
    B21 = np.array([[0.0, 0.0],[1.0, 0.0]])

    d11 = _fd_dir_p_out(beta_u, beta_v, gamma, delta, y, z, Q, B11, eps)
    d22 = _fd_dir_p_out(beta_u, beta_v, gamma, delta, y, z, Q, B22, eps)
    d12 = _fd_dir_p_out(beta_u, beta_v, gamma, delta, y, z, Q, B12, eps)
    d21 = _fd_dir_p_out(beta_u, beta_v, gamma, delta, y, z, Q, B21, eps)

    G = np.array([[d11, d12],
                  [d21, d22]])
    return G

# --- wrapper: build c1,c2 and compare analytic vs numeric ---

def test_partial_Q_p_out_once(beta_u, beta_v, gamma, delta, y, z, Q, eps=2e-4):
    # compute c1, c2 from your imported helpers
    bt = beta_tilde(beta_v)        # = beta_v/(1+beta_v+sqrt(1+beta_v))
    c2 = c_2(bt, Q[1,1])
    c1 = c_1(c2, bt)

    # analytic gradient (your function)
    G_an = partial_Q_p_out(beta_u, beta_v, gamma, delta, y, z, Q, bt, c1, c2)

    # numeric (finite differences)
    G_fd = _numeric_grad_wrt_Q(beta_u, beta_v, gamma, delta, y, z, Q, eps=eps)
    G_fd[1,0] = G_fd[0,1] ## need to understand better derivatives wrt symmetric matrices, right now this

    diff = G_an - G_fd
    rel = np.divide(diff, np.maximum(1e-10, np.abs(G_fd)), where=np.ones_like(G_fd,dtype=bool))

    print("z =", z)
    print("Q =\n", Q)
    print("\n∂_Q p_out (analytic):\n", G_an)
    print("∂_Q p_out (FD):\n", G_fd)
    print("diff (an - fd):\n", diff)
    print("rel. diff:\n", rel)
    print("Frobenius |diff| =", np.linalg.norm(diff, 'fro'))
    return G_an, G_fd, diff

# ---- Example usage ----
np.random.seed(1)
beta_u, beta_v = 1.0, 2.0
gamma, delta   = 1.0, 0.0

# pick a random symmetric positive definite Q and a test z
Q = np.array([[0.8, 0.3], [0.3, 0.7]])
z = np.random.randn(2) * 0.2

print("=== y = +1 ===")
test_partial_Q_p_out_once(beta_u, beta_v, gamma, delta, y=1, z=z, Q=Q, eps=2e-4)


=== y = +1 ===
z = [ 0.32486907 -0.12235128]
Q =
 [[0.8 0.3]
 [0.3 0.7]]

∂_Q p_out (analytic):
 [[-0.03416312 -0.20518069]
 [-0.20518069 -0.10868513]]
∂_Q p_out (FD):
 [[-0.03416312 -0.20518069]
 [-0.20518069 -0.10868513]]
diff (an - fd):
 [[1.55154375e-10 2.87858051e-09]
 [2.87858051e-09 3.94007729e-10]]
rel. diff:
 [[4.54157451e-09 1.40294903e-08]
 [1.40294903e-08 3.62522189e-09]]
Frobenius |diff| = 4.092892193640042e-09


(array([[-0.03416312, -0.20518069],
        [-0.20518069, -0.10868513]]),
 array([[-0.03416312, -0.20518069],
        [-0.20518069, -0.10868513]]),
 array([[1.55154375e-10, 2.87858051e-09],
        [2.87858051e-09, 3.94007729e-10]]))

Testing fQ

In [16]:
from tqdm import tqdm

In [54]:
def f_Q(beta_u, beta_v, gamma, delta, beta_tilde_v, c1, c2, y, omega, Q, M, N, V, a, b, c, d, fout, dfout):
    fQ = np.zeros((2,2))

    if y == -1:
        return fQ
    
    if y == 1:
        omega1, omega2 = omega

        int1 = int_1(a, b, c, d, gamma, delta)
        int2 = int_2(a, b, c, d, gamma, delta)
        int3 = int_3(a, b, c, d, gamma, delta)
        int4 = int_4(a, b, c, d, gamma, delta)
        int5 = int_5(a, b, c, d, gamma, delta)

        fQ[0,0] = - 1/2 * beta_u * int5 / int1
        fQ[0,1] = -1/int1*(np.sqrt(beta_u*beta_v)/c2*int4 + beta_u*c1*Q[0,1]*int5 - np.sqrt(beta_u)*c1*(np.sqrt(beta_u)*(M[0,1]+c1*Q[0,1]*M[1,1])*int5 + np.sqrt(beta_v)/c2*M[1,1]*int4 + (1- M[1,1]*c1)*omega2*int2))
        fQ[1,0] = fQ[0,1]
        #fQ11
        A = -1/2*beta_v + beta_tilde_v/c2 
        C = beta_tilde_v*np.sqrt(beta_v)/c2**2*(np.sqrt(beta_u)*(M[0,1]+c1*Q[0,1]*M[1,1])*int4 + np.sqrt(beta_v)/c2*M[1,1]*int1 + (1- M[1,1]*c1)*omega2*int3)
        D = np.sqrt(beta_u)*beta_tilde_v**2/c2**2*(1+2/c2)*Q[0,1]*(np.sqrt(beta_u)*(M[0,1]+c1*Q[0,1]*M[1,1])*int5+np.sqrt(beta_v)/c2*M[1,1]*int4+omega2*(1-M[1,1]*c1)*int2)
        E = - beta_tilde_v/c2**2 * np.sqrt(beta_u*beta_v)*Q[0,1]*int4
        F = - 1/2 * beta_u * beta_tilde_v**2/c2**2 * (1+2/c2)*Q[0,1]**2 * int5

        B1 = int1*(M[1,1]-(N[0,1]*omega1+N[1,1]*omega2)**2-M[1,1]**2*beta_v/c2**2)
        B2 = - beta_u*(M[0,1]+c1*Q[0,1]*M[1,1])**2*int5
        B3 = - 2*np.sqrt(beta_u*beta_v)/c2*(M[0,1]+c1*Q[0,1]*M[1,1])*M[1,1]*int4
        B4 = - 2*(N[0,1]*omega1+N[1,1]*omega2)*np.sqrt(beta_u)*(M[0,1]+c1*Q[0,1]*M[1,1])*int2
        B5 = - 2*(N[0,1]*omega1+N[1,1]*omega2)*np.sqrt(beta_v)/c2*M[1,1]*int3

        # B = -1/2*beta_tilde_v**2/c2**2*(1+2/c2)*(B1 + B2 + B3 + B4 + B5)
        covZ = np.outer(omega + V @ fout, omega + V @ fout) + V @ dfout @ V + V
        B = - 1/2 * beta_tilde_v**2/c2**2*(1+2/c2)*covZ[1,1]

        #fQ[1,1] = A + 1/int1*(B + C + D + E + F)
        fQ[1,1] = A + B + 1/int1*(C + D + E + F)

        return fQ

In [57]:


# ---------- shared param builder (same structure/style as earlier) ----------
def _build_params_Q(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    bt = beta_tilde(beta_v)                      # beta_tilde_v
    c2 = c_2(bt, Q[1,1])
    c1 = c_1(c2, bt)

    Gamma = gamma_matrix(c1)                     # 2x2, only [1,1]=c1
    Vinv  = np.linalg.inv(V)
    M     = np.linalg.inv(Vinv + Gamma)
    N     = Vinv @ M

    omega1, omega2 = omega
    a, b, c, d = integral_parameters(beta_u, beta_v, omega, Q, N, M, c1, c2)

    # your confirmed analytic Z_out for the denominator:
    Zout_an = Z_out(beta_v, gamma, delta, c1, c2, y, omega, Q, M, N, a, b, c, d)
    g_an = f_out(beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)
    J_an = df_out(g_an, beta_u, beta_v, gamma, delta, Gamma, c1, c2, y, omega, Q, N, a, b, c, d)

    

    return bt, c1, c2, M, N, a, b, c, d, Vinv, Zout_an, Gamma, g_an, J_an


# ---------- Monte Carlo f_Q using analytic Z_out in the denominator ----------
def fQ_MC_using_ZoutAnalytic(beta_u, beta_v, gamma, delta, y, omega, Q, V,
                             n_samples=200_000, n_boot=100, eps=0.0):
    """
    Returns:
      fQ_mc: 2x2 MC estimate of f_Q = E[∂_Q P_out]/Z_out
      SE:    2x2 bootstrap SE (if n_boot>0) else None
    """
    bt, c1, c2, M, N, a, b, c, d, Vinv, Zout_an, Gamma, fout, dfout = _build_params_Q(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )

    # Draw Z ~ N(omega, V)
    Z = np.random.multivariate_normal(mean=omega, cov=V, size=n_samples)

    # Accumulate E[∂_Q P_out] with your analytic partial_Q_p_out
    acc = np.zeros((2,2))
    for i in tqdm(range(n_samples)):
        acc += partial_Q_p_out(beta_u, beta_v, gamma, delta, y, Z[i], Q, bt, c1, c2)
    dP_dQ_mean = acc / n_samples

    fQ_mc = dP_dQ_mean / (Zout_an + 1e-15)

    return fQ_mc


# ---------- Analytic f_Q wrapper (calls your f_Q exactly as given) ----------
def fQ_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V):
    bt, c1, c2, M, N, a, b, c, d, Vinv, Zout_an, Gamma, fout, dfout = _build_params_Q(
        beta_u, beta_v, gamma, delta, y, omega, Q, V
    )
    # your signature: f_Q(..., beta_tilde_v, c1, c2, ..., M, N, a,b,c,d)
    return f_Q(beta_u, beta_v, gamma, delta, bt, c1, c2, y, omega, Q, M, N, V, a, b, c, d, fout, dfout)


# ---------- One-shot comparison (like before) ----------
def test_fQ_once(beta_u=1.0, beta_v=2.0, gamma=0.0, delta=1.0, y=1,
                 Q=None, V=None, omega=None, n=200_000, boot=100):
    if Q is None:
        A = np.random.randn(2,2); Q = A.T @ A + 0.3*np.eye(2)
    if V is None:
        B = np.random.randn(2,2); V = B.T @ B + 0.3*np.eye(2)
    if omega is None:
        omega = np.array([0.15, -0.05])

    fQ_mc = fQ_MC_using_ZoutAnalytic(beta_u, beta_v, gamma, delta, y, omega, Q, V,
                                         n_samples=n, n_boot=boot)
    fQ_an = fQ_analytic(beta_u, beta_v, gamma, delta, y, omega, Q, V)

    print(f"y={y}")
    print("f_Q  (MC using analytic Z_out):")
    print(fQ_mc)
    
    print("f_Q  (analytic):")
    print(fQ_an)
    print("diff (MC - analytic):")
    print(fQ_mc - fQ_an)

    frob = np.linalg.norm(fQ_mc - fQ_an, 'fro')
    print(f"Frobenius diff: {frob:.4e}\n")


# ---------- Example with your Q, V ----------
np.random.seed(1)

beta_u, beta_v = 1.0, 2.0
gamma, delta   = 1.0, 0.0
omega = np.random.randn(2) * 0.2

Q = np.array([[0.6, 0.3],
              [0.3, 2]])
V = np.array([[0.2, -0.3],
              [-0.3, 0.5]])

y = 1
test_fQ_once(beta_u, beta_v, gamma, delta, y,
                 Q=Q, V=V, omega=omega, n=200_000, boot=100)


100%|██████████| 200000/200000 [00:10<00:00, 18313.58it/s]

y=1
f_Q  (MC using analytic Z_out):
[[-0.1875806  -0.8038184 ]
 [-0.8038184   1.32247438]]
f_Q  (analytic):
[[-0.18722825 -0.80564955]
 [-0.80564955  1.32671299]]
diff (MC - analytic):
[[-0.00035236  0.00183115]
 [ 0.00183115 -0.00423861]]
Frobenius diff: 4.9796e-03

